# Draft `pyGIMLi(emg3d)` - Inversion

**An attempt at using `pyGIMLi` as an inversion framework for `emg3d` computations.**

=> Create an environment using the provided `environment.yml`: `conda env create`.

For developing purposes, we take a very simple model and survey:
- Simple double-halfspace model water-subsurface with a resistive block.
- Survey: A single 2D line, 7 sources, 7 frequencies.

**Current Limitations**
- Only isotropic models supported
- Without el. perm. and magn. perm.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import emg3d
import pygimli as pg

## Load survey (incl. data), initial model, and create a Simulation

In [ ]:
inp_data = emg3d.load('pginv-7x7.h5')        # Finer model (but still coarse)

inp_survey = inp_data['survey']
inp_model = inp_data['model']
inp_grid = inp_model.grid
inp_model

In [ ]:
# The model is resistivity. Change it to conductivity
# TODO: make this internally happen, so that pyGIMLi
# always gets a conductivity model!
con_model = emg3d.Model(inp_grid, 1/inp_model.property_x, mapping='Conductivity')
con_model

In [ ]:
# For dev-purposes, we only select one source
# [for dev-purposes it is also a very simple model]
#inp_survey = inp_survey.select(sources='TxED-1')
inp_survey

## Create a Starting Model

In [ ]:
hx = np.ones(58)*250.0
hy = np.ones(26)*250.0
hz = np.ones(15)*250.0
grid = emg3d.TensorMesh([hx, hy, hz], [-7250, -3250, -3500])

model = emg3d.Model(grid, 1.0, mapping='Conductivity')
model.property_x[:, :, -1:] = 3.33

# QC
grid

## Create a Simulation

In [ ]:
# Create an emg3d Simulation instance
sim = emg3d.simulations.Simulation(
    survey=inp_survey,
    model=model,
    gridding='both', #'same',  # I would like to make that more flexible in the future
    gridding_opts={'vector': 'xyz'},
    max_workers=50,    # Adjust as needed
    receiver_interpolation='linear',  # Currently necessary for the gradient
    #solver_opts={'plain': True, 'maxit': 1},  # Just for dev-purpose
    #solver_opts={'tol': 1e-4},                # Just for dev-purpose
    tqdm_opts=False,  # Switch off verbose progress bars
)
sim

## Instantiate and run inversion

In [ ]:
markers = np.zeros(sim.model.shape, dtype=int)
markers[1:-1, :, :][:, 1:-1, :][:, :, 1:-2] = 1

fop = emg3d.inversion.pygimli.Kernel(simulation=sim, markers=markers, pgthreads=1)

INV = emg3d.inversion.pygimli.Inversion(fop=fop)

INV.setRegularization(1, limits=(0.001, 2), startModel=1)
INV.setRegularization(0, background=True)

In [ ]:
#fop.obs_errors[fop.obs_errors > 0.5] = 1e8
invmodel = INV.run(
    #maxIter=4, # just to test
    lam=1,
    #verbose=True,
    #startModel=...,
    #isReference=True,
)

In [ ]:
emg3d.save('result.h5', model=invmodel)

## Plots

In [ ]:
emg3d.Report()